In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
                .master("local[2]")\
                .config('spark.executor.memory','450M')\
                .config('spark.sql.warehouse.dir',"./warehouse")\
                .config('spark.local.directory','./shuffle')\
                .config('spark.sql.files.maxPartitionBytes','128M')\
                .appName('test.com')\
                .getOrCreate()

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.host', '10.88.51.138'),
 ('spark.driver.port', '53124'),
 ('spark.app.name', 'test.com'

In [4]:
dim_listing = spark.read\
    .option("header","true")\
    .option("inferSchema", "true")\
    .csv("./data/Listings.csv")

dim_listing.show()

+----------+--------------------+--------+----------+--------------------+------------------+------------------+--------------------+-----------------+-------------------------+--------------------+----------------------+-------------------+--------+-----+--------+---------+----------------+------------+------------+--------+--------------------+---------------+---------------+---------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+--------------------+--------------------+
|listing_id|                name| host_id|host_since|       host_location|host_response_time|host_response_rate|host_acceptance_rate|host_is_superhost|host_total_listings_count|host_has_profile_pic|host_identity_verified|      neighbourhood|district| city|latitude|longitude|   property_type|   room_type|accommodates|bedrooms|           amenities|          price| minimum_nights| maximum_nights|review_scores_rating|r

In [5]:
fact_review = spark.read\
    .option("header","true")\
    .option("inferSchema", "true")\
    .csv("./data/Reviews.csv")

fact_review.show()

+----------+---------+----------+-----------+
|listing_id|review_id|      date|reviewer_id|
+----------+---------+----------+-----------+
|     11798|330265172|2018-09-30|   11863072|
|     15383|330103585|2018-09-30|   39147453|
|     16455|329985788|2018-09-30|    1125378|
|     17919|330016899|2018-09-30|  172717984|
|     26827|329995638|2018-09-30|   17542859|
|     74561|330089224|2018-09-30|  173044789|
|    140355|330194958|2018-09-30|  160093807|
|    162163|329980859|2018-09-30|   94026758|
|    167998|329950677|2018-09-30|   35388162|
|    178188|330213008|2018-09-30|    3652511|
|    232956|330140226|2018-09-30|   57664145|
|    266725|330038354|2018-09-30|   77387165|
|    314288|330299075|2018-09-30|  192717587|
|    325880|330081449|2018-09-30|  154527568|
|    335003|329968377|2018-09-30|    3461699|
|    348747|330131287|2018-09-30|    9554201|
|    352851|330201364|2018-09-30|  142182690|
|    378714|330246144|2018-09-30|   15772951|
|    406852|330283854|2018-09-30| 

In [6]:
dim_listing.write.saveAsTable('dim_listing')

dim_listing.rdd.getNumPartitions()

2

In [7]:
spark.sql("""SELECT count(*) FROM dim_listing""").show()

+--------+
|count(1)|
+--------+
|  279712|
+--------+



In [8]:
dim_listing.createOrReplaceTempView('listing')
fact_review.createOrReplaceTempView('review')

In [9]:
query = """SELECT * FROM listing inner join review as r_1 on listing.listing_id=r_1.listing_id
                                        join review as r_2 on r_1.listing_id=r_2.listing_id"""

df = spark.sql(query)

df.collect()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\envs\myenv\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "c:\Users\Admin\anaconda3\envs\myenv\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\envs\myenv\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\Admin\anaconda3\envs\myenv\lib\site-packages\py4j\clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it